## Installation

In [1]:
%%time

! pip install huggify-data

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 90.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 93.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

CPU times: user 560 ms, sys: 112 ms, total: 672 ms
Wall time: 1min 38s


## Data

In [ ]:
from datasets import load_dataset

ds = load_dataset("Aiwensile2/StorySeed")

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/4376 [00:00<?, ? examples/s]

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['prompt', 'response'],
        num_rows: 4376
    })
})

In [ ]:
ds['train']['prompt'][0]

'Please help me create a picture book with the theme of\nA curious kitten embarks on a whimsical journey through a magical forest, encountering friendly creatures along the way.\nThe format should include a title, followed by a cover page with an illustration only, and the remaining pages should each have both text and illustrations.'

In [ ]:
ds['train']['response'][0]

'### Title: Whisker\'s Magical Forest Adventure\n\n---\n\n### Cover Page Illustration:\nA small, fluffy, gray kitten with wide, curious eyes standing at the edge of a lush, vibrant forest filled with colorful flowers and twinkling lights.\n\n---\n\n### Page 1 Illustration:\nWhiskers peeking out from behind a large oak tree, gazing at the path ahead.\n\n#### Text:\nIn the heart of a sun-dappled garden, little Whiskers the kitten found a hidden path leading into a mysterious forest. With a twitch of his nose and a flick of his tail, he decided it was the perfect day for an adventure.\n\n---\n\n### Page 2 Illustration:\nWhiskers meeting a cheerful squirrel with a bushy tail, sitting on a tree branch.\n\n#### Text:\nNot far into the forest, Whiskers met Squeaky the squirrel. "Where are you off to, little kitten?" Squeaky chirped. "I\'m exploring this magical forest," Whiskers replied with a purr. Squeaky nodded, "Follow me! I\'ll show you the way to the Whispering Willows."\n\n---\n\n### P

```python
### Human: What does YSA stand for? ### Assistant: YSA stands for Youth Spirit Artworks.
```

In [ ]:
from tqdm import tqdm

In [ ]:
questions = []
answers = []
texts = []

for i in tqdm(range(100)):
    ques = ds['train']['prompt'][i]
    answ = ' '.join(ds['train']['response'][i].split('### '))
    text = "### Human: " + ques + " ### Assistant: " + answ

    questions.append(ques)
    answers.append(answ)
    texts.append(text)

100%|██████████| 100/100 [00:02<00:00, 44.47it/s]


In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame({'questions': questions, 'answers': answers, 'text': texts})
df.head()

,questions,answers,text
0,Please help me create a picture book with the ...,Title: Whisker's Magical Forest Adventure\n\n...,### Human: Please help me create a picture boo...
1,Please help me create a picture book with the ...,Title: **Mia and Mico’s Treasure Quest**\n\n ...,### Human: Please help me create a picture boo...
2,Please help me create a picture book with the ...,Title: The Enchanted Doorway\n\n---\n\n# Cove...,### Human: Please help me create a picture boo...
3,Please help me create a picture book with the ...,"Title: ""The Cave of Dragon Friends""\n\n---\n\...",### Human: Please help me create a picture boo...
4,Please help me create a picture book with the ...,Title: Pip's Quest for the Golden Cheese\n\n ...,### Human: Please help me create a picture boo...


In [ ]:
df.to_csv('data.csv', index=False)

## Training

In [ ]:
from google.colab import userdata

In [ ]:
from huggify_data.push_modules import DataFrameUploader

# Example usage:
df = pd.read_csv('/content/data.csv')
uploader = DataFrameUploader(
    df,
    hf_token=userdata.get('HF_TOKEN'),
    repo_name='storySeed-v3', # this is whatever you want
    username='eagle0504' # your own user name
)
uploader.process_data()
uploader.push_to_hub()

Dataframe verified: columns 'questions' and 'answers' are present.
Data processed into DatasetDict.
Repository created: eagle0504/storySeed-v3


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Dataset pushed to Hugging Face Hub.


In [ ]:
from huggify_data.train_modules import *

In [ ]:
# Usage
model_name = "NousResearch/Llama-2-7b-chat-hf"
dataset_name = "eagle0504/storySeed-v3"
new_model = "storySeed-v3-llama2"
huggingface_token = userdata.get('HF_TOKEN')

In [ ]:
%%time

trainer = LlamaTrainer(model_name, dataset_name, new_model, huggingface_token)
peft_config = trainer.configure_lora()
training_args = trainer.configure_training_arguments(num_train_epochs=1)
trainer.train_model(training_args, peft_config)

## Inference

In [1]:
from huggify_data.train_modules import *
from google.colab import userdata

In [2]:
%%time

# Usage
model_name = "NousResearch/Llama-2-7b-chat-hf"
dataset_name = "eagle0504/storySeed-v3"
new_model = "storySeed-v3-llama2"
huggingface_token = userdata.get('HF_TOKEN')
trainer = LlamaTrainer(model_name, dataset_name, new_model, huggingface_token)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

Model loading completed.
CPU times: user 22.8 s, sys: 21.8 s, total: 44.6 s
Wall time: 2min 44s


In [3]:
some_model, some_tokenizer = trainer.load_model_and_tokenizer(
    base_model_path="NousResearch/Llama-2-7b-chat-hf",
    new_model_path="eagle0504/storySeed-v3-llama2",
)

adapter_config.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

In [4]:
# Example usage
prompt = "### Human: Please help me create a picture book with the theme of A curious kitten embarks on a whimsical journey through a magical forest, encountering friendly creatures along the way. The format should include a title, followed by a cover page with an illustration only, and the remaining pages should each have both text and illustrations. ### Assistant:"
response = trainer.generate_response(some_model, some_tokenizer, prompt, 300)
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


### Human: Please help me create a picture book with the theme of A curious kitten embarks on a whimsical journey through a magical forest, encountering friendly creatures along the way. The format should include a title, followed by a cover page with an illustration only, and the remaining pages should each have both text and illustrations. ### Assistant: Of course, I'd be happy to help you create a picture book! Here's a rough outline of the format you requested: Title: "Whiskers in the Woods" Cover Page: Title in large font on a plain, white background, with a beautiful illustration of a kitten perched on a branch, surrounded by trees with wispy leaves and bright, vibrant flowers Cover Page Illustration: A green landscape with a tall tree and branches that stretch to the moon. Illustration: 1-5 Pages: "The Journey Begins" - 200 words, one image of a kitten standing in front of a large door, with its ears perked up and paws ready to go 4-5 images of the kitten stepping through the do

In [6]:
print(response.split("### Assistant: ")[1])

Of course, I'd be happy to help you create a picture book! Here's a rough outline of the format you requested: Title: "Whiskers in the Woods" Cover Page: Title in large font on a plain, white background, with a beautiful illustration of a kitten perched on a branch, surrounded by trees with wispy leaves and bright, vibrant flowers Cover Page Illustration: A green landscape with a tall tree and branches that stretch to the moon. Illustration: 1-5 Pages: "The Journey Begins" - 200 words, one image of a kitten standing in front of a large door, with its ears perked up and paws ready to go 4-5 images of the kitten stepping through the door and walking into the woods, each image with a different expression on the kitten's face, such as curious, excited, or surprised 6-7 Pages: "The Forest's Wonders" - 200 words, one image of the kitten
